Import des librairies et du modele d'entrainement puis de la fonction d'extraction du texte


In [113]:
import spacy
import pdfplumber
from spacy.training.example import Example
import json
import re


def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text(encoding='utf-8')  # Spécifie l'encodage
    return text

def extract_phone_numbers(text):
    phone_numbers = re.findall(r'\b\d{2}[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{2}[-.\s]?\d{2}\b', text)
    return phone_numbers

def extract_emails(text):
    emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    return emails
def extract_languages(text):
    languages = re.findall(r'\b(?:français|anglais|espagnol|allemand)\b(?:\s(?:A|B|C)\d)?', text, flags=re.IGNORECASE)
    return languages

def extract_date_ranges(text):
    date_ranges = []
    # Expressions régulières pour les séparateurs '-' et '_'
    regex_dash = r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}\s-\s(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}'
    regex_underscore = r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}\s_\s(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}'
    regex_underscore_witout_space = r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}_(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}'
    regex_dash_witout_space = r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}-(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}'
    date_rangeM =r'(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4}\s–\s(?:Maintenant|(?:Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre)\s\d{4})'

    # Recherche des plages de dates dans le texte
    matches_dash = re.findall(regex_dash, text)
    matches_underscore = re.findall(regex_underscore, text)
    matches_underscore_witout_space = re.findall(regex_underscore_witout_space, text)
    matches_dash_witout_space = re.findall(regex_dash_witout_space, text)
    matches_date_rangeM = re.findall(date_rangeM, text)
    # Ajout des résultats des deux expressions régulières dans un seul tableau
    date_ranges.extend(matches_underscore_witout_space)
    date_ranges.extend(matches_underscore)
    date_ranges.extend(matches_dash)
    date_ranges.extend(matches_dash_witout_space)
    date_ranges.extend(matches_date_rangeM)

    return date_ranges

Affichage du texte extrait

In [114]:
pdf_path = "CV-14.pdf"  # Remplacez ceci par votre chemin
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

Mélisande Yung
Responsable marketing
3 ans chez Rakuten, en tant que chef de projet marketing e-commerce et assistante
marketing (secteur mode et beauté). J’y ai augmenté le nombre de visiteurs de 30% et passé
le taux de conversion de 1,8% à 4,2%. Maîtrisant les supports du marketing digital, réseaux
sociaux et contenu (CMS et Google Analytics), je peux permettre à votre boutique de mode
en ligne d’exploser en nombre de visiteurs et d’acheteur
Experiences professionnelles
Décembre 2017 – Maintenant
Rakuten France, Paris
Chef de projet marketing e-commerce (secteur mode et beauté)
Coordonnées
• Développer le trafic vers la boutique en ligne Mode et Beauté (+30% trafic).
• Augmenter le taux de conversion (+133%) et réduire le taux de rebond (-85%).
• Superviser les équipes marketing et Interface Utilisateur pour améliorer le site et le
Adresse
processus d’achat.
• Planifier le budget des stratégies et projets webmarketing.
15, boulevard Amiral
Courbet
Novembre 2016 – Décembre 2017
Rakute

Code pour entrainer l'analyseur de PDF

In [115]:
# Lecture des données annotées pour l'entraînement du modèle
with open('train_data.json', 'r', encoding='utf-8') as file:
    training_data = json.load(file)

# Entraînement du modèle SpaCy
nlp = spacy.blank("fr")
ner = nlp.add_pipe("ner", last=True)

# Ajout des labels
ner.add_label("PER")
ner.add_label("EXP")
ner.add_label("DATE")
# Ajoute d'autres labels nécessaires (PHONE, EMAIL, etc.)






# Création des exemples pour l'entraînement du modèle
examples = []
for data in training_data:
    text = data["text"]
    entities = []
    for entity in data["entities"]:
        start, end, label = entity["start"], entity["end"], entity["label"]
        entities.append((start, end, label))
    examples.append(Example.from_dict(nlp.make_doc(text), {"entities": entities}))

# Entraînement du modèle
nlp.initialize(lambda: examples)
for i in range(10):  # Nombre d'itérations d'entraînement
    losses = {}
    nlp.update(examples, drop=0.5, losses=losses)
    print(losses)

# Sauvegarde du modèle entraîné
nlp.to_disk("modele_cv")

{'ner': 31.571430206298828}
{'ner': 33.08907747268677}
{'ner': 31.74453830718994}
{'ner': 31.57049548625946}
{'ner': 30.441071689128876}
{'ner': 33.05082646012306}
{'ner': 33.48699629306793}
{'ner': 34.50110548734665}
{'ner': 34.516260266304016}
{'ner': 32.75094082951546}


Test du modele d'entrainement 

In [116]:
# Chargement du modèle SpaCy entraîné
nlp_loaded = spacy.load("modele_cv")

# Extraction des informations du nouveau CV
pdf_path = "CV-14.pdf"
cv_text = extract_text_from_pdf(pdf_path)
doc = nlp_loaded(cv_text)

# Affichage des entités extraites du CV
for ent in doc.ents:
    print(ent.text, ent.label_)


text extraction mail et telephone

In [117]:
phone_numbers = extract_phone_numbers(extracted_text)
emails = extract_emails(extracted_text)

print("Numéros de téléphone trouvés :")
print(phone_numbers)
print("\nAdresses e-mail trouvées :")
print(emails)
print("\nLangues parlées :")
print(extract_languages(extracted_text))


dates = extract_date_ranges(extracted_text)
print("Plages de dates trouvées :", dates)

Numéros de téléphone trouvés :
['0485435365']

Adresses e-mail trouvées :
['nom@gmail.com']

Langues parlées :
['Français', 'Anglais']


NameError: name 'date_rangesss' is not defined